<a href="https://colab.research.google.com/github/palxx/_100_days_of_CUDA/blob/main/cuda_day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile vector_multiplication.cu

Writing vector_multiplication.cu


In [3]:
%%writefile vector_multiplication.cu
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>
#include <chrono>

void printMatrix(const float *Matrix, const int size = 16) {
    int rootSize = sqrt(size);
    for (int i = 0; i < rootSize; i++) {
        for (int j = 0; j < rootSize; j++) {
            std::cout << Matrix[i * rootSize + j] << " ";
        }
        std::cout << "\n";
    }
}

__global__ void matrixAddCUDA(const float *Matrix_A, const float *Matrix_B, float *Matrix_C,
                              const int sizeX, const int sizeY) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (row < sizeY && col < sizeX) {
        Matrix_C[row * sizeX + col] = Matrix_A[row * sizeX + col] + Matrix_B[row * sizeX + col];
    }
}

void matrixAddCPU(const float *Matrix_A, const float *Matrix_B, float *Matrix_C, int sizeX, int sizeY) {
    for (int row = 0; row < sizeY; row++) {
        for (int col = 0; col < sizeX; col++) {
            Matrix_C[row * sizeX + col] = Matrix_A[row * sizeX + col] + Matrix_B[row * sizeX + col];
        }
    }
}

void compareExecutionTime(const float *Matrix_A, const float *Matrix_B, float *Matrix_C,
                          const int sizeX, const int sizeY) {
    const int matrixSize = sizeX * sizeY;
    const int matrixBytes = sizeof(float) * matrixSize;

    float *gpu_A, *gpu_B, *gpu_C;
    cudaMalloc((void **)&gpu_A, matrixBytes);
    cudaMalloc((void **)&gpu_B, matrixBytes);
    cudaMalloc((void **)&gpu_C, matrixBytes);

    cudaMemcpy(gpu_A, Matrix_A, matrixBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_B, Matrix_B, matrixBytes, cudaMemcpyHostToDevice);

    int BLOCK_SIZE = 32;
    dim3 blockDim(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridDim((sizeX + BLOCK_SIZE - 1) / BLOCK_SIZE, (sizeY + BLOCK_SIZE - 1) / BLOCK_SIZE);

    auto startCPU = std::chrono::high_resolution_clock::now();
    matrixAddCPU(Matrix_A, Matrix_B, Matrix_C, sizeX, sizeY);
    auto endCPU = std::chrono::high_resolution_clock::now();

    auto startCUDA = std::chrono::high_resolution_clock::now();
    matrixAddCUDA<<<gridDim, blockDim>>>(gpu_A, gpu_B, gpu_C, sizeX, sizeY);
    cudaDeviceSynchronize();
    auto endCUDA = std::chrono::high_resolution_clock::now();

    cudaMemcpy(Matrix_C, gpu_C, matrixBytes, cudaMemcpyDeviceToHost);

    std::chrono::duration<double> cpuDuration = endCPU - startCPU;
    std::chrono::duration<double> cudaDuration = endCUDA - startCUDA;
    std::cout << "CPU Execution Time: " << cpuDuration.count() << " seconds\n";
    std::cout << "CUDA Execution Time: " << cudaDuration.count() << " seconds\n";

    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);
}

int main() {
    const int sizeX = 1024*16;
    const int sizeY = 1024*16;
    const int matrixSize = sizeX * sizeY;

    float *cpu_A = new float[matrixSize];
    float *cpu_B = new float[matrixSize];
    float *cpu_C = new float[matrixSize];

    for (int i = 0; i < matrixSize; i++) {
        cpu_A[i] = 10.0f;
        cpu_B[i] = static_cast<float>(i);
    }

    compareExecutionTime(cpu_A, cpu_B, cpu_C, sizeX, sizeY);

    delete[] cpu_A;
    delete[] cpu_B;
    delete[] cpu_C;

    return 0;
}


Overwriting vector_multiplication.cu


In [4]:
!nvcc vector_multiplication.cu -o vector_multiplication

In [5]:
!./vector_multiplication

CPU Execution Time: 1.36273 seconds
CUDA Execution Time: 0.0525025 seconds


In [6]:
import triton
import torch
import triton.language as tl


@triton.jit
def addMatrix(Matrix_A,Matrix_B,Matrix_C,sizeX,sizeY,BLOCK_SIZE:tl.constexpr):

    pid_x = tl.program_id(0) # we have the rows
    pid_y = tl.program_id(1) # we have the collumns

    row_start = pid_x*BLOCK_SIZE
    col_start = pid_y*BLOCK_SIZE

    row_indices = row_start + tl.arange(0,BLOCK_SIZE)
    col_indices = col_start + tl.arange(0,BLOCK_SIZE)

    row_indices = row_indices[:,None]
    col_indices = col_indices[None,:]

    row_mask = row_indices < sizeY
    col_mask = col_indices < sizeX
    valid_mask = row_mask & col_mask

    flat_indicies = row_indices * sizeX + col_indices

    A = tl.load(Matrix_A + flat_indicies,mask =valid_mask,other=0.0)
    B = tl.load(Matrix_B + flat_indicies,mask = valid_mask,other = 0.0)

    C = A+B;

    tl.store(Matrix_C+flat_indicies,C,mask=valid_mask)


def test_addMatrix():
    sizeX = 8
    sizeY = 8
    BLOCK_SIZE = 2

    Matrix_A = torch.randn(sizeY, sizeX, device='cuda', dtype=torch.float32)
    Matrix_B = torch.randn(sizeY, sizeX, device='cuda', dtype=torch.float32)
    Matrix_C = torch.zeros_like(Matrix_A, device='cuda', dtype=torch.float32)

    Matrix_A_flat = Matrix_A.flatten()
    Matrix_B_flat = Matrix_B.flatten()
    Matrix_C_flat = Matrix_C.flatten()

    grid = (triton.cdiv(sizeX, BLOCK_SIZE), triton.cdiv(sizeY, BLOCK_SIZE))
    addMatrix[grid](Matrix_A_flat, Matrix_B_flat, Matrix_C_flat, sizeX, sizeY, BLOCK_SIZE)

    Matrix_C = Matrix_C_flat.reshape(sizeY, sizeX)

    expected = Matrix_A + Matrix_B
    print("Matrix A:\n", Matrix_A)
    print("Matrix B:\n", Matrix_B)
    print("Matrix C (Triton):\n", Matrix_C)
    print("Expected (PyTorch):\n", expected)
    assert torch.allclose(Matrix_C, expected), "Triton result does not match PyTorch result!"

test_addMatrix()


Matrix A:
 tensor([[-2.3228,  0.2932, -2.7785, -0.0496,  0.8685, -0.5088,  0.0826,  0.1319],
        [ 1.1190, -0.7372, -0.4857,  1.2607, -0.4451, -0.3621, -0.0861, -0.1779],
        [ 1.0730, -0.6525,  0.5174, -0.9272, -1.8129,  0.4633,  0.6948,  0.7564],
        [ 1.1736,  1.8948, -0.2438,  0.0666, -0.4462, -0.7321, -0.3145, -0.2999],
        [-0.1199, -0.9977, -1.6680,  0.3026, -1.5908, -1.6825, -1.7616,  0.9824],
        [ 0.0869, -1.1895,  1.2076, -0.4781,  0.3477, -0.9023, -0.9323, -1.1272],
        [ 0.2496,  0.0708, -0.4251,  0.5966, -0.7472, -1.4395, -0.4526, -0.1734],
        [ 1.2109, -0.5930,  0.2552, -0.4246, -0.1997, -1.3577, -0.3124, -0.5672]],
       device='cuda:0')
Matrix B:
 tensor([[-0.7049,  0.9256,  0.8780,  0.7826,  0.4499, -0.2101,  0.8249, -0.3770],
        [ 0.8817,  0.2678,  0.0624, -0.4547, -1.4823, -2.1206,  0.6962,  0.1820],
        [-0.9694, -0.5023,  0.0289,  0.2932,  1.1516,  1.4994, -0.2897, -0.3089],
        [-0.0764, -0.7051,  0.5688,  0.6604,  2.564